In [1]:
import numpy as np
from datetime import datetime, timedelta
start_time = datetime.now().replace(microsecond=0)

In [2]:
np.random.seed(1991)

In [3]:
from stonesoup.models.transition.linear import CombinedLinearGaussianTransitionModel, ConstantVelocity
from stonesoup.types.groundtruth import GroundTruthPath, GroundTruthState

transition_model = CombinedLinearGaussianTransitionModel([ConstantVelocity(0.05),
                                                          ConstantVelocity(0.05)])
timesteps = [start_time]
truth = GroundTruthPath([GroundTruthState([0, 1, 0, 1], timestamp=start_time)])
for k in range(1, 21):
    timesteps.append(start_time + timedelta(seconds=k))
    truth.append(GroundTruthState(transition_model.function(truth[k-1], noise=True, time_interval=timedelta(seconds=1)), timestamp=timesteps[k]))

In [4]:
from stonesoup.plotter import AnimatedPlotterly
plotter = AnimatedPlotterly(timesteps, tail_length=1)
plotter.plot_ground_truths(truth, [0, 2])
plotter.fig

In [5]:
from stonesoup.models.measurement.nonlinear import CartesianToBearingRange
sensor_x = 50
sensor_y = 0

measurement_model = CartesianToBearingRange(
    ndim_state = 4,
    mapping = (0,2),
    noise_covar = np.diag([np.radians(0.2), 1]),
    translation_offset = np.array([[sensor_x], [sensor_y]])
)

In [6]:
from stonesoup.types.detection import Detection
measurements = []
for state in truth:
    measurement = measurement_model.function(state, noise = True)
    measurements.append(Detection(measurement, timestamp = state.timestamp, measurement_model = measurement_model))

In [7]:
plotter.plot_measurements(measurements, [0,2])
plotter.fig

In [8]:
from stonesoup.predictor.kalman import ExtendedKalmanPredictor
predictor = ExtendedKalmanPredictor(transition_model)

from stonesoup.updater.kalman import ExtendedKalmanUpdater
updater = ExtendedKalmanUpdater(measurement_model)

In [9]:
from stonesoup.types.state import GaussianState
prior = GaussianState([[0], [1], [0], [1]], np.diag([1.5, 0.5, 1.5, 0.5]), timestamp=start_time)

In [10]:
from stonesoup.types.hypothesis import SingleHypothesis
from stonesoup.types.track import Track
track = Track()

for measurement in measurements:
    prediction = predictor.predict(prior, timestamp = measurement.timestamp)
    hypothesis = SingleHypothesis(prediction, measurement) # group a prediction and a measurement
    post = updater.update(hypothesis)
    track.append(post)
    prior = track[-1]


In [11]:
plotter.plot_tracks(track, [0, 2], uncertainty=True)
plotter.fig